In [1]:
# !pip install deeppavlov emoji
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
# !pip install tensorflow==2.15.1
# !pip install transformers
# !export PATH=/usr/local/cuda-12.4/bin:$PATH
# !export LD_LIBRARY_PATH=/usr/local/cuda-12.4/lib64:$LD_LIBRARY_PATH

In [2]:
# Настройка предупреждений
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [3]:
import os
import gdown
import nltk
import subprocess
import cudf.pandas
cudf.pandas.install()
import pandas as pd
import locale
import torch
from transformers import pipeline
import spacy
from torch.utils.data import DataLoader, Dataset
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import tensorflow as tf
from collections import defaultdict, Counter
from textblob import TextBlob
import re
from string import punctuation
from spacy.lang.ru.stop_words import STOP_WORDS as stopwords_ru
from sklearn.feature_extraction.text import CountVectorizer
from deeppavlov import build_model, configs
from itertools import combinations
import warnings
from nltk.corpus import stopwords
from tqdm import tqdm
import emoji

2024-08-01 09:47:13.513137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 09:47:13.566966: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 09:47:13.583511: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-01 09:47:13.692616: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-01 09:47:14.735239: W tensorflow/compiler/tf2

In [4]:
cuda_available = torch.cuda.is_available()
num_gpus = torch.cuda.device_count()
devices = tf.config.list_physical_devices('GPU')
gpu_available = len(devices) > 0

print(f"CUDA Available: {cuda_available}")
print(f"Number of GPUs: {num_gpus}")
print(f"GPU Available: {gpu_available}")
for device in devices:
    print(f"Device: {device}")

CUDA Available: True
Number of GPUs: 1
GPU Available: False


I0000 00:00:1722505636.507535      39 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-01 09:47:16.515358: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
import os
import gdown

def download_file_if_not_exists(file_url, output_path):
    """Скачивает файл с Google Drive, если он ещё не существует в указанной директории."""
    # Проверка наличия файла
    if os.path.exists(output_path):
        print(f"Файл '{output_path}' уже существует.")
    else:
        print(f"Файл '{output_path}' не найден. Начинаю загрузку...")
        gdown.download(file_url, output_path, quiet=False)
        print(f"Файл '{output_path}' успешно загружен.")

# Указываем URL и путь к файлу
file_url = 'https://drive.google.com/uc?id=1alondqI-2IHo__mYU7KQz4Ip8ytYGHXg'
output_file_name = 'wildberries_reviews.csv'  # Укажите реальное имя файла, которое хотите сохранить
output_path = os.path.join(os.getcwd(), output_file_name)  # Полный путь к файлу

download_file_if_not_exists(file_url, output_path)


Файл '/workspace/wildberries_reviews.csv' уже существует.


In [6]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

# Загрузка необходимых ресурсов nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
import subprocess

def install_spacy_model_if_not_exists(model_name):
    """Устанавливает модель spaCy, если она ещё не установлена."""
    try:
        # Получаем список установленных моделей
        result = subprocess.run(
            ['python', '-m', 'spacy', 'info'],
            capture_output=True, text=True, check=True
        )
        
        # Проверяем наличие модели в списке установленных
        if model_name in result.stdout:
            print(f"Модель {model_name} уже установлена.")
        else:
            print(f"Модель {model_name} не найдена. Устанавливаю...")
            subprocess.run(['python', '-m', 'spacy', 'download', model_name], check=True)
            print(f"Модель {model_name} успешно установлена.")
    
    except subprocess.CalledProcessError as e:
        print(f"Ошибка при выполнении команды: {e}")

# Замените 'ru_core_news_lg' на нужное имя модели
install_spacy_model_if_not_exists('ru_core_news_lg')


Модель ru_core_news_lg уже установлена.


In [8]:
import cudf.pandas
cudf.pandas.install()
import pandas as pd

In [9]:
# Загрузка модели для русского языка
spacy.prefer_gpu()
nlp = spacy.load('ru_core_news_lg')

In [10]:
df_raw = pd.read_csv('wildberries_reviews.csv', nrows=5000)

In [11]:
df_raw.head()

,Unnamed: 0,review_full_text,review_rating,product,category,url,corrected_text
0,0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Работает хорошо.
1,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и..."
2,2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ..."
3,3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...
4,4,"Всё в комплекте, есть инструкция на русском яз...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Всё в комплекте, есть инструкция на русском яз..."


In [12]:
df_test = df_raw.drop(["Unnamed: 0"], axis=1)

In [13]:
df_test.groupby("category").count().sort_values(by="review_full_text", ascending=False)

,review_full_text,review_rating,product,url,corrected_text
category,,,,,
/Автотовары/Автокосметика и автохимия,2195,2195,2195,2195,2195
/Автотовары/Автокосметика и автохимия/Очистители автомобильные,1253,1253,1253,1253,1253
/Автотовары/OFFroad,703,703,703,703,703
/Автотовары/Автокосметика и автохимия/Присадки,472,472,472,472,472
/Автотовары/Автокосметика и автохимия/Герметики автомобильные,377,377,377,377,377


In [14]:
# Настройка устройства (CPU или GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [15]:
# df = df_test[df_test['category'].str.contains('/Дом/Кухня/Кухонный текстиль')]
# df = df_test[df_test["category"] == "/Для ремонта/Инструменты и оснастка/Монтажные инструменты"]
# df = df_test[df_test["product"] == "VIRA / Клещи переставные 250 мм"]
df = df_test

In [16]:
" ".join(df["review_full_text"].to_list())

'Работает хорошо. Пришло быстро, все целое на вид. Завтра буду испытывать.. Купил на квадр для поднятия отвала, установка заняла час👍 Лебёдка хорошая. Но в инструкции ни слова про сборку и крепление к машине. Всё в комплекте, есть инструкция на русском языке. Муж ещё не пробовал 👍 Пока при проверке работает нормально, а там посмотрим на сколько хватит Не комплект, нет крюка, заметил поздно, кроме возврата ничего не предложили, проверяйте комплектность внимательнее, а не так как я. При первой поездки в лес, барабан отломился когда начал намотку, а после и вообще трос зажувала! Деньги на ветер! Мужу заказала. Товар понравился. Только не доложили в комплекте одну гайку. Установил, работает. Полный комплект для установки. установили. сразу начала подозрительно хрустеть без груза. второй раз включил ее попробовать, всё, уже не отпускает. только поднимает. при нажатии на кнопку отпускания, провод задымел. сейчас будем через тех поддержку решать как то вопрос. цена не три копейки Все в компле

In [17]:
!pip install tf-keras

In [18]:
import re
import pandas as pd
from string import punctuation
from spacy.lang.ru.stop_words import STOP_WORDS as stopwords_ru
from sklearn.feature_extraction.text import CountVectorizer
from transformers import pipeline

# Создаём pipeline для анализа тональности с помощью transformers
sentiment_pipeline = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment", device=device)


In [19]:
df_raw = df

In [20]:
import re
import pandas as pd
from string import punctuation
from spacy.lang.ru import Russian
from spacy.pipeline import Sentencizer
from transformers import pipeline
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor

stopwords_ru = stopwords.words("russian")
black_list = ["спасибо", "доставка", "продавец", "покупка", "хороший", "отличный", "очень", "прийти"] + list(stopwords_ru)
optimal_batch_size = 256

In [21]:
import re
import emoji

def clean_text(text):
    # Функция для удаления эмодзи
    def remove_emoji(text):
        return emoji.replace_emoji(text, replace='')

    text = remove_emoji(text)
    return text

In [22]:
# Проверка доступности GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [23]:
import spacy
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# Инициализация spaCy
spacy.prefer_gpu()
nlp = spacy.load("ru_core_news_lg")


In [24]:
def get_segments(doc):
    """Извлекает смысловые конструкции из документа без исправления орфографии."""
    segments = []
    segment_data = []
    segment = []

    for sentence in doc.sents:
        for token in sentence:
            if token.is_punct:
                if token.text in {'.', '!', '?'}:
                    if segment:
                        segments.append({
                            'text': ' '.join(segment),
                            'details': segment_data
                        })
                        segment = []
                        segment_data = []
                else:
                    if segment:
                        segment.append(token.text)
                        segment_data.append({
                            'text': token.text,
                            'lemma': token.lemma_,
                            'pos': token.pos_,
                            'dep': token.dep_
                        })
            else:
                segment.append(token.text)
                segment_data.append({
                    'text': token.text,
                    'lemma': token.lemma_,
                    'pos': token.pos_,
                    'dep': token.dep_
                })

    if segment:
        segments.append({
            'text': ' '.join(segment),
            'details': segment_data
        })

    return segments

def get_word_constructions(doc):
    """Извлекает конструкции связанных слов из документа."""
    constructions = []

    for sentence in doc.sents:
        for token in sentence:
            if token.dep_ in {'amod', 'nsubj', 'obj', 'obl'}:
                head = token.head
                if head.pos_ in {'NOUN', 'VERB'}:
                    construction = [head.text]
                    construction_data = [{
                        'text': head.text,
                        'lemma': head.lemma_,
                        'pos': head.pos_,
                        'dep': head.dep_
                    }]
                    for child in head.children:
                        if child.dep_ in {'amod', 'case', 'det', 'advmod', 'obl'}:
                            construction.append(child.text)
                            construction_data.append({
                                'text': child.text,
                                'lemma': child.lemma_,
                                'pos': child.pos_,
                                'dep': child.dep_
                            })
                    constructions.append({
                        'text': ' '.join(construction),
                        'details': construction_data
                    })
    
    return constructions

def process_texts(texts, batch_size):
    """Обрабатывает список текстов, разбивая их на сегменты и конструкции связанных слов."""
    segments = []
    constructions = []
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = []
        for doc in tqdm(nlp.pipe(texts, batch_size=batch_size), total=len(texts)):
            futures.append(executor.submit(get_segments, doc))
            futures.append(executor.submit(get_word_constructions, doc))
        for i, future in enumerate(futures):
            if i % 2 == 0:
                segments.append(future.result())
            else:
                constructions.append(future.result())
    return segments, constructions

def preprocess_and_explode(df, batch_size):
    """Предварительная обработка текста и развертывание DataFrame."""

    texts = df["corrected_text"].apply(clean_text).tolist()
    processed_segments, processed_constructions = process_texts(texts, batch_size)

    df["processed_segments"] = processed_segments
    df["processed_constructions"] = processed_constructions

    # Разворачиваем колонки с сегментами и конструкциями на уровне строк
    df_exploded_segments = df.explode("processed_segments").reset_index(drop=True)
    df_exploded_constructions = df.explode("processed_constructions").reset_index(drop=True)

    return df_exploded_segments, df_exploded_constructions

# Применение функций
df_exploded_segments, df_exploded_constructions = preprocess_and_explode(df, batch_size=2048)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:15<00:00, 314.71it/s]


In [29]:
# Разворачиваем столбец processed_constructions на два столбца
df_exploded_constructions[["text", "text_details"]] = df_exploded_constructions["processed_constructions"].apply(lambda x: pd.Series(x) if isinstance(x, dict) else pd.Series({"text": None, "details": None}))["text", "text_details"]
df_details = df_exploded_constructions.drop(["processed_constructions"], axis=1)
df_details

,review_full_text,review_rating,product,category,url,corrected_text,processed_segments,text_details,text
0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Работает хорошо.,"[{'details': [{'dep': 'ROOT', 'lemma': 'работа...",None,None
1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...","[{'details': [{'dep': 'ROOT', 'lemma': 'прийти...",None,None
2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","[{'details': [{'dep': 'ROOT', 'lemma': 'купить...",Купил квадр час,"[{'dep': 'ROOT', 'lemma': 'купить', 'pos': 'VE..."
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","[{'details': [{'dep': 'ROOT', 'lemma': 'купить...",заняла,"[{'dep': 'conj', 'lemma': 'занять', 'pos': 'VE..."
4,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","[{'details': [{'dep': 'ROOT', 'lemma': 'купить...",Купил квадр час,"[{'dep': 'ROOT', 'lemma': 'купить', 'pos': 'VE..."
...,...,...,...,...,...,...,...,...,...
16333,Доставка быстрая. Заказываем не в первый раз. ...,5,FAZZH-1 Жидкая гильзовка / Присадка в масло дв...,/Автотовары/Автокосметика и автохимия/Присадки,https://www.wildberries.ru/catalog/90516640/fe...,Доставка быстрая. Заказываем не в первый раз. ...,"[{'details': [{'dep': 'nsubj', 'lemma': 'доста...",раз не в первый,"[{'dep': 'obl', 'lemma': 'раз', 'pos': 'NOUN',..."
16334,Доставка быстрая. Заказываем не в первый раз. ...,5,FAZZH-1 Жидкая гильзовка / Присадка в масло дв...,/Автотовары/Автокосметика и автохимия/Присадки,https://www.wildberries.ru/catalog/90516640/fe...,Доставка быстрая. Заказываем не в первый раз. ...,"[{'details': [{'dep': 'nsubj', 'lemma': 'доста...",Заказываем раз,"[{'dep': 'ROOT', 'lemma': 'заказывать', 'pos':..."
16335,Классная вещь уничтожила мотор,1,FAZZH-1 Жидкая гильзовка / Присадка в масло дв...,/Автотовары/Автокосметика и автохимия/Присадки,https://www.wildberries.ru/catalog/90516640/fe...,Классная вещь уничтожила мотор,"[{'details': [{'dep': 'amod', 'lemma': 'классн...",вещь Классная,"[{'dep': 'nsubj', 'lemma': 'вещь', 'pos': 'NOU..."
16336,Классная вещь уничтожила мотор,1,FAZZH-1 Жидкая гильзовка / Присадка в масло дв...,/Автотовары/Автокосметика и автохимия/Присадки,https://www.wildberries.ru/catalog/90516640/fe...,Классная вещь уничтожила мотор,"[{'details': [{'dep': 'amod', 'lemma': 'классн...",уничтожила,"[{'dep': 'ROOT', 'lemma': 'уничтожить', 'pos':..."


: 

In [25]:
# Разворачиваем столбец processed_text на два столбца
df_exploded[["text_details", "text"]] = df_exploded["processed_text"].apply(lambda x: pd.Series(x) if isinstance(x, dict) else pd.Series({"text": None, "details": None}))
df_details = df_exploded.drop(["processed_text"], axis=1)

NameError: name 'df_exploded' is not defined

In [58]:
df_details

,review_full_text,review_rating,product,category,url,corrected_text,text_details,text
0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Работает хорошо.,"[{'dep': 'ROOT', 'lemma': 'работать', 'pos': '...",Работает хорошо
1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...","[{'dep': 'ROOT', 'lemma': 'прийти', 'pos': 'VE...","Пришло быстро , все целое на вид"
2,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...","[{'dep': 'advmod', 'lemma': 'завтра', 'pos': '...",Завтра буду испытывать
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","[{'dep': 'ROOT', 'lemma': 'купить', 'pos': 'VE...","Купил на квадр для поднятия отвала , установка..."
4,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,"[{'dep': 'nsubj', 'lemma': 'лебёдка', 'pos': '...",Лебёдка хорошая
...,...,...,...,...,...,...,...,...
9188,муж остался довольным,5,FAZZH-1 Жидкая гильзовка / Присадка в масло дв...,/Автотовары/Автокосметика и автохимия/Присадки,https://www.wildberries.ru/catalog/90516640/fe...,Муж остался довольным,"[{'dep': 'nsubj', 'lemma': 'муж', 'pos': 'NOUN...",Муж остался довольным
9189,Доставка быстрая. Заказываем не в первый раз. ...,5,FAZZH-1 Жидкая гильзовка / Присадка в масло дв...,/Автотовары/Автокосметика и автохимия/Присадки,https://www.wildberries.ru/catalog/90516640/fe...,Доставка быстрая. Заказываем не в первый раз. ...,"[{'dep': 'nsubj', 'lemma': 'доставка', 'pos': ...",Доставка быстрая
9190,Доставка быстрая. Заказываем не в первый раз. ...,5,FAZZH-1 Жидкая гильзовка / Присадка в масло дв...,/Автотовары/Автокосметика и автохимия/Присадки,https://www.wildberries.ru/catalog/90516640/fe...,Доставка быстрая. Заказываем не в первый раз. ...,"[{'dep': 'ROOT', 'lemma': 'заказывать', 'pos':...",Заказываем не в первый раз
9191,Доставка быстрая. Заказываем не в первый раз. ...,5,FAZZH-1 Жидкая гильзовка / Присадка в масло дв...,/Автотовары/Автокосметика и автохимия/Присадки,https://www.wildberries.ru/catalog/90516640/fe...,Доставка быстрая. Заказываем не в первый раз. ...,"[{'dep': 'nsubj', 'lemma': 'муж', 'pos': 'NOUN...",Муж доволен


In [60]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.cluster import KMeans

# Функция для извлечения депендентов и частей речи
def extract_features(details):
    if details is None:
        return {}
    features = {}
    for detail in details:
        dep = detail['dep']
        pos = detail['pos']
        if dep not in features:
            features[dep] = 0
        if pos not in features:
            features[pos] = 0
        features[dep] += 1
        features[pos] += 1
    return features

# Применим функцию к каждому элементу в колонке text_details
df_details['features'] = df_details['text_details'].apply(extract_features)

# Преобразуем словари с признаками в формат, пригодный для машинного обучения
vectorizer = DictVectorizer(sparse=False)
X = vectorizer.fit_transform(df_details['features'])

# Применим алгоритм кластеризации
kmeans = KMeans(n_clusters=2, random_state=42)
df_details['cluster'] = kmeans.fit_predict(X)

# Показать результат
df_details[['text', 'features', 'cluster']]


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# Функция для проверки наличия существительных в записи
def contains_noun(text_details):
    if isinstance(text_details, list):
        return any(token['pos'] == 'NOUN' for token in text_details)
    return False

# Фильтрация датафрейма
df_details[df_details['text_details'].apply(contains_noun)]

,review_full_text,review_rating,product,category,url,corrected_text,text_details,text
1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...","[{'dep': 'ROOT', 'lemma': 'прийти', 'pos': 'VE...","Пришло быстро , все целое на вид"
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","[{'dep': 'ROOT', 'lemma': 'купить', 'pos': 'VE...","Купил на квадр для поднятия отвала , установка..."
5,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,"[{'dep': 'cc', 'lemma': 'но', 'pos': 'CCONJ', ...",Но в инструкции ни слова про сборку и креплени...
6,"Всё в комплекте, есть инструкция на русском яз...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Всё в комплекте, есть инструкция на русском яз...","[{'dep': 'parataxis', 'lemma': 'всё', 'pos': '...","Всё в комплекте , есть инструкция на русском я..."
7,Муж ещё не пробовал,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Муж ещё не пробовал,"[{'dep': 'nsubj', 'lemma': 'муж', 'pos': 'NOUN...",Муж ещё не пробовал
...,...,...,...,...,...,...,...,...
9188,муж остался довольным,5,FAZZH-1 Жидкая гильзовка / Присадка в масло дв...,/Автотовары/Автокосметика и автохимия/Присадки,https://www.wildberries.ru/catalog/90516640/fe...,Муж остался довольным,"[{'dep': 'nsubj', 'lemma': 'муж', 'pos': 'NOUN...",Муж остался довольным
9189,Доставка быстрая. Заказываем не в первый раз. ...,5,FAZZH-1 Жидкая гильзовка / Присадка в масло дв...,/Автотовары/Автокосметика и автохимия/Присадки,https://www.wildberries.ru/catalog/90516640/fe...,Доставка быстрая. Заказываем не в первый раз. ...,"[{'dep': 'nsubj', 'lemma': 'доставка', 'pos': ...",Доставка быстрая
9190,Доставка быстрая. Заказываем не в первый раз. ...,5,FAZZH-1 Жидкая гильзовка / Присадка в масло дв...,/Автотовары/Автокосметика и автохимия/Присадки,https://www.wildberries.ru/catalog/90516640/fe...,Доставка быстрая. Заказываем не в первый раз. ...,"[{'dep': 'ROOT', 'lemma': 'заказывать', 'pos':...",Заказываем не в первый раз
9191,Доставка быстрая. Заказываем не в первый раз. ...,5,FAZZH-1 Жидкая гильзовка / Присадка в масло дв...,/Автотовары/Автокосметика и автохимия/Присадки,https://www.wildberries.ru/catalog/90516640/fe...,Доставка быстрая. Заказываем не в первый раз. ...,"[{'dep': 'nsubj', 'lemma': 'муж', 'pos': 'NOUN...",Муж доволен


In [ ]:
df_details["text_details"][0]

[{'dep': 'ROOT', 'lemma': 'работать', 'pos': 'VERB', 'text': 'Работает'},
 {'dep': 'advmod', 'lemma': 'хорошо', 'pos': 'ADV', 'text': 'хорошо'}]

In [ ]:
def extract_compound_nouns_and_lemmas(text_details):
    if not isinstance(text_details, list):
        return [], []

    nouns = []
    lemmas = []
    i = 0
    length = len(text_details)
    
    while i < length:
        token = text_details[i]
        
        if token['pos'] == 'NOUN':
            # Составные существительные через дефис
            if (i + 2 < length and 
                text_details[i + 1]['text'] == '-' and 
                text_details[i + 2]['pos'] == 'NOUN'):
                compound_noun = f"{token['text']}-{text_details[i + 2]['text']}"
                compound_lemma = f"{token['lemma']}-{text_details[i + 2]['lemma']}"
                nouns.append(compound_noun)
                lemmas.extend([token['lemma'], '-', text_details[i + 2]['lemma']])
                i += 3  # Пропустить следующие два токена
            # Составные существительные через пробел (или с пробелом перед следующим существительным)
            elif (i + 1 < length and 
                  text_details[i + 1]['pos'] == 'NOUN'):
                compound_noun = f"{token['text']} {text_details[i + 1]['text']}"
                compound_lemma = f"{token['lemma']} {text_details[i + 1]['lemma']}"
                nouns.append(compound_noun)
                lemmas.extend([token['lemma'], text_details[i + 1]['lemma']])
                i += 2  # Пропустить следующий токен
            else:
                nouns.append(token['text'])
                lemmas.append(token['lemma'])
                i += 1
        else:
            # Добавляем служебные части речи к леммам, если они между существительными
            lemmas.append(token['lemma'])
            i += 1

    return nouns, lemmas

# Применение функции к DataFrame
df_details[['nouns', 'lemmas']] = df_details['text_details'].apply(lambda x: pd.Series(extract_compound_nouns_and_lemmas(x)))

# Эксплодируем датафрейм по существительным
df_details_exploded = df_details.explode('nouns')

# Удаляем строки, где нет существительных
df_details_exploded = df_details_exploded[df_details_exploded['nouns'].notna()]

df_details_exploded

,review_full_text,review_rating,product,category,url,corrected_text,text_details,text,nouns,lemmas
1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...","[{'dep': 'ROOT', 'lemma': 'прийти', 'pos': 'VE...","Пришло быстро , все целое на вид",вид,"[прийти, быстро, ,, все, целый, на, вид]"
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","[{'dep': 'ROOT', 'lemma': 'купить', 'pos': 'VE...","Купил на квадр для поднятия отвала , установка...",квадр,"[купить, на, квадр, для, поднятие, отвал, ,, у..."
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","[{'dep': 'ROOT', 'lemma': 'купить', 'pos': 'VE...","Купил на квадр для поднятия отвала , установка...",поднятия отвала,"[купить, на, квадр, для, поднятие, отвал, ,, у..."
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","[{'dep': 'ROOT', 'lemma': 'купить', 'pos': 'VE...","Купил на квадр для поднятия отвала , установка...",установка,"[купить, на, квадр, для, поднятие, отвал, ,, у..."
3,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","[{'dep': 'ROOT', 'lemma': 'купить', 'pos': 'VE...","Купил на квадр для поднятия отвала , установка...",час,"[купить, на, квадр, для, поднятие, отвал, ,, у..."
...,...,...,...,...,...,...,...,...,...,...
9189,Доставка быстрая. Заказываем не в первый раз. ...,5,FAZZH-1 Жидкая гильзовка / Присадка в масло дв...,/Автотовары/Автокосметика и автохимия/Присадки,https://www.wildberries.ru/catalog/90516640/fe...,Доставка быстрая. Заказываем не в первый раз. ...,"[{'dep': 'nsubj', 'lemma': 'доставка', 'pos': ...",Доставка быстрая,Доставка,"[доставка, быстрый]"
9190,Доставка быстрая. Заказываем не в первый раз. ...,5,FAZZH-1 Жидкая гильзовка / Присадка в масло дв...,/Автотовары/Автокосметика и автохимия/Присадки,https://www.wildberries.ru/catalog/90516640/fe...,Доставка быстрая. Заказываем не в первый раз. ...,"[{'dep': 'ROOT', 'lemma': 'заказывать', 'pos':...",Заказываем не в первый раз,раз,"[заказывать, не, в, первый, раз]"
9191,Доставка быстрая. Заказываем не в первый раз. ...,5,FAZZH-1 Жидкая гильзовка / Присадка в масло дв...,/Автотовары/Автокосметика и автохимия/Присадки,https://www.wildberries.ru/catalog/90516640/fe...,Доставка быстрая. Заказываем не в первый раз. ...,"[{'dep': 'nsubj', 'lemma': 'муж', 'pos': 'NOUN...",Муж доволен,Муж,"[муж, довольный]"
9192,Классная вещь уничтожила мотор,1,FAZZH-1 Жидкая гильзовка / Присадка в масло дв...,/Автотовары/Автокосметика и автохимия/Присадки,https://www.wildberries.ru/catalog/90516640/fe...,Классная вещь уничтожила мотор,"[{'dep': 'amod', 'lemma': 'классный', 'pos': '...",Классная вещь уничтожила мотор,вещь,"[классный, вещь, уничтожить, мотор]"


In [ ]:
# Применяем функцию для объединения текста и text_details
def combine_text_and_details(group):
    combined_text = '. '.join(group['text'])
    combined_details = [item for sublist in group['text_details'] for item in sublist]
    return pd.Series([combined_text, combined_details], index=['text', 'text_details'])

# Группируем по продукту и существительным, объединяем текст и text_details
grouped_text = df_details_exploded.groupby(['product', 'nouns']).apply(combine_text_and_details).reset_index()
# Фильтрация строк, где длина текста больше 50 символов
grouped_text = grouped_text[grouped_text['text'].apply(len) > 50]

temp = grouped_text


In [ ]:
grouped_text

,product,nouns,text,text_details
2,3TON / Очиститель карбюратора и дроссельной за...,Крышка,Крышка на очистителе расколота и держится толь...,"[{'dep': 'nsubj', 'lemma': 'крышка', 'pos': 'N..."
9,3TON / Очиститель карбюратора и дроссельной за...,Спасибо,"Спасибо за товар и быструю доставку. Спасибо ,...","[{'dep': 'ROOT', 'lemma': 'спасибо', 'pos': 'N..."
10,3TON / Очиститель карбюратора и дроссельной за...,Супер,"Супер. Супер , объем хороший , форсунки отличн...","[{'dep': 'ROOT', 'lemma': 'супер', 'pos': 'NOU..."
11,3TON / Очиститель карбюратора и дроссельной за...,Товар,Товар пока не проверяли. Товар соответствует о...,"[{'dep': 'obj', 'lemma': 'товар', 'pos': 'NOUN..."
16,3TON / Очиститель карбюратора и дроссельной за...,Чистит,Чистит отлично. Чистит хорошо. Чистит. Чистит ...,"[{'dep': 'nsubj', 'lemma': 'чистит', 'pos': 'N..."
...,...,...,...,...
9337,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,проблем,"Все отлично , подошли без проблем. Все встало ...","[{'dep': 'nsubj', 'lemma': 'все', 'pos': 'PRON..."
9338,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,профи,"На вид не плохие , но к сожалению на уаз профи...","[{'dep': 'case', 'lemma': 'на', 'pos': 'ADP', ..."
9340,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,см.,"Все встало без проблем , уаз поднялся на 3 см....","[{'dep': 'nsubj', 'lemma': 'все', 'pos': 'PRON..."
9341,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,сожалению,"На вид не плохие , но к сожалению на уаз профи...","[{'dep': 'case', 'lemma': 'на', 'pos': 'ADP', ..."


In [ ]:
import pandas as pd
from collections import Counter
from tqdm import tqdm

# Заменяем grouped_text на temp
grouped_text = temp

# Функция для обработки одной строки DataFrame
def process_row(row):
    adj_lemmas = []
    previous_word = None

    for detail in row['text_details']:
        lemma = detail['lemma']
        pos = detail['pos']

        # Исключение пунктуации
        if pos == 'PUNCT':
            continue

        # Объединение с частицей "не"
        if previous_word == 'не':
            lemma = 'не' + lemma
            previous_word = None
        elif lemma == 'не':
            previous_word = 'не'
            continue

        if lemma not in stopwords_ru:  # Исключение стоп-слов
            # Добавление только прилагательных
            if pos == 'ADJ':
                adj_lemmas.append(lemma)

    # Подсчет лемм (только прилагательные)
    lemma_counts = Counter(adj_lemmas)
    filtered_lemma_counts = [
        (lemma, count)
        for lemma, count in lemma_counts.items()
        if count > 1  # Убираем значения <= 1
    ]
    # Сортировка по убыванию count
    filtered_lemma_counts.sort(key=lambda x: x[1], reverse=True)

    return {'lemma_counts': filtered_lemma_counts if filtered_lemma_counts else None}

# Применение функции с отображением прогресса
tqdm.pandas(desc="Processing rows")
grouped_text_results = grouped_text.progress_apply(process_row, axis=1)

# Преобразование результатов в отдельные колонки
results_df = pd.DataFrame(grouped_text_results.tolist())
grouped_text = grouped_text.join(results_df)

# Удаление строк с пустыми значениями в 'lemma_counts'
grouped_text = grouped_text.dropna(subset=['lemma_counts'], how='all')

Processing rows: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6664/6664 [00:01<00:00, 5408.91it/s]


In [ ]:
grouped_text

,product,nouns,text,text_details,lemma_counts
26,3TON / Очиститель карбюратора и дроссельной за...,генераторе,"Чистил карбюратор на генераторе , всё отчистил...","[{'dep': 'ROOT', 'lemma': 'чистил', 'pos': 'VE...","[(хороший, 3)]"
47,3TON / Очиститель карбюратора и дроссельной за...,норм,"Спасибо , всё норм. Очищает норм , хватило поч...","[{'dep': 'ROOT', 'lemma': 'спасибо', 'pos': 'N...","[(едкий, 2)]"
63,3TON / Очиститель карбюратора и дроссельной за...,спасибо,"Хороший очиститель , чистил дроссель старой То...","[{'dep': 'amod', 'lemma': 'хороший', 'pos': 'A...","[(большой, 3)]"
65,3TON / Очиститель карбюратора и дроссельной за...,средство,"Муж сказал , что средство работает отлично. От...","[{'dep': 'nsubj', 'lemma': 'муж', 'pos': 'NOUN...","[(хороший, 3)]"
85,AG TECH / Антискотч спрей удалитель наклеек и ...,Итог,"Итог на фото , за 2 часа окно стало как новое ...","[{'dep': 'nsubj', 'lemma': 'итог', 'pos': 'NOU...","[(отличный, 2), (липкий, 2)]"
...,...,...,...,...,...
6611,Shtapler / Лебедка электрическая 12v 3500lb 15...,ура,"Нет , со своей задачей эта лебёдка справляется...","[{'dep': 'parataxis', 'lemma': 'нет', 'pos': '...","[(самое, 2)]"
6613,Shtapler / Лебедка электрическая 12v 3500lb 15...,функционалом,Лебёдку использую для поднятия и опускания сне...,"[{'dep': 'obj', 'lemma': 'лебёдка', 'pos': 'NO...","[(плохой, 2)]"
6621,Shtapler / Лебедка электрическая 12v 4500lb 20...,Проверка,"Проверка прошла успешно , буду дальше эксплуат...","[{'dep': 'nsubj', 'lemma': 'проверка', 'pos': ...","[(плохой, 4)]"
6623,Shtapler / Лебедка электрическая 12v 4500lb 20...,Продавцу респект,"Продавцу респект и уважу ха , однозначно реком...","[{'dep': 'iobj', 'lemma': 'продавец', 'pos': '...","[(плохой, 2)]"


In [ ]:
# Функция для вычисления суммы значений в lemma_counts
def sum_lemma_counts(lemma_counts):
    if isinstance(lemma_counts, dict):
        # Используем 0 для значений, которые могут быть None
        return sum(count for count in lemma_counts.values() if count is not None)
    return 0

# Применение функции к DataFrame для создания новой колонки
grouped_text['lemma_counts_sum'] = grouped_text['lemma_counts'].apply(sum_lemma_counts)

# Сортировка DataFrame по суммарному количеству в колонке 'lemma_counts_sum'
grouped_text_sorted = grouped_text.sort_values(by='lemma_counts_sum', ascending=False)

# Удаление колонки 'lemma_counts_sum', если не нужна после сортировки
grouped_text_sorted = grouped_text_sorted.drop(columns=['lemma_counts_sum'])
grouped_text_sorted

,product,nouns,text,text_details,lemma_counts
26,3TON / Очиститель карбюратора и дроссельной за...,генераторе,"Чистил карбюратор на генераторе , всё отчистил...","[{'dep': 'ROOT', 'lemma': 'чистил', 'pos': 'VE...","[(хороший, 3)]"
5120,GRASS / Универсальный пенный очиститель Multip...,пункте выдачи,На пункте выдачи заметила только треснувший ко...,"[{'dep': 'case', 'lemma': 'на', 'pos': 'ADP', ...","[(мелкий, 2)]"
5013,GRASS / Универсальный пенный очиститель Multip...,коллеги,"Товар пришел быстро , целый , по рекомендациям...","[{'dep': 'nsubj', 'lemma': 'товар', 'pos': 'NO...","[(мелкий, 2)]"
5022,GRASS / Универсальный пенный очиститель Multip...,крышка треснута,Впервые не проверила целостность на ПВЗ и вот ...,"[{'dep': 'advmod', 'lemma': 'впервые', 'pos': ...","[(негативный, 2), (неплохой, 2), (мягкий, 2), ..."
5033,GRASS / Универсальный пенный очиститель Multip...,максимуму,"Баллона хватило на всю машину , но я в несколь...","[{'dep': 'nsubj', 'lemma': 'баллона', 'pos': '...","[(первый, 4), (похожий, 2), (второй, 2), (порш..."
...,...,...,...,...,...
3036,"GRASS / Автошампунь для бесконтактной мойки, A...",автомобиль,На втором фото после еще дополнительно протира...,"[{'dep': 'case', 'lemma': 'на', 'pos': 'ADP', ...","[(отличный, 8), (хороший, 7), (приятный, 2)]"
3034,"GRASS / Автошампунь для бесконтактной мойки, A...",авто шампуня,"Вместо авто шампуня , положили ополаскиватель ...","[{'dep': 'case', 'lemma': 'вместо', 'pos': 'AD...","[(нормальный, 2)]"
3017,"GRASS / Автошампунь для бесконтактной мойки, A...",Развел,Развел 50/50 и в отличие от картеровского шамп...,"[{'dep': 'nsubj:pass', 'lemma': 'развел', 'pos...","[(задний, 2), (тормозной, 2)]"
3015,"GRASS / Автошампунь для бесконтактной мойки, A...",Производителю респект,\n Последние время WB очень расстраивает .......,"[{'dep': 'dep', 'lemma': ' ', 'pos': 'SPACE',...","[(хороший, 4)]"


In [ ]:
# Функция для подсчета суммы количества лемм для каждого существительного
def calculate_lemma_sum(lemma_counts):
    if isinstance(lemma_counts, list):
        return sum(count for lemma, count in lemma_counts)
    return 0

# Добавляем новый столбец с суммой количества лемм для каждого существительного
grouped_text_sorted['lemma_sum'] = grouped_text_sorted['lemma_counts'].apply(calculate_lemma_sum)

# Сортируем DataFrame по убыванию суммы количества лемм
grouped_text_sorted_sorted = grouped_text_sorted.sort_values(by='lemma_sum', ascending=False)
grouped_text_sorted_sorted

,product,nouns,text,text_details,lemma_counts,lemma_sum
6138,LAVR / Трехуровневая очистка топливной системы...,двигатель,"По ощущениям действительно показалось , что дв...","[{'dep': 'case', 'lemma': 'по', 'pos': 'ADP', ...","[(полный, 4), (вкусный, 4), (поляронная, 4), (...",52
5972,"LAVR / Кондиционер кожи Восстанавливающий, 255 мл",спасибо продавцу,"На запах конечно не очень , но работает доволь...","[{'dep': 'case', 'lemma': 'на', 'pos': 'ADP', ...","[(приятный, 13), (отличный, 5), (стойкий, 3), ...",49
406,AG TECH / Антискотч спрей удалитель наклеек и ...,помощи,\n Хотел оттереть следы скотча с торпеды при п...,"[{'dep': 'dep', 'lemma': ' ', 'pos': 'SPACE', ...","[(отличный, 9), (хороший, 7), (пластиковый, 5)...",40
6292,LIQUI MOLY / Герметик радиатора 1997 250мл,бмв,"Залил в бмв е 39 , течь прекратилась через пол...","[{'dep': 'ROOT', 'lemma': 'залить', 'pos': 'VE...","[(отличный, 6), (обалденный, 6), (довольный, 4...",38
4932,GRASS / Универсальный пенный очиститель Multip...,Сиденья,"Сиденья чистые теперь , были сильные разводы. ...","[{'dep': 'nsubj', 'lemma': 'сидение', 'pos': '...","[(общий, 5), (рабочий, 4), (моторный, 4), (мас...",35
...,...,...,...,...,...,...
6014,LAVR / Очиститель шлема и визора MOTO 20 мл,поездку,"Баночка маленькая , компактная , очень удобно ...","[{'dep': 'nsubj', 'lemma': 'баночка', 'pos': '...","[(задний, 2)]",2
6102,LAVR / Трехуровневая очистка топливной системы...,Магистраль,Магистраль была забита поднятыми мягкими отлож...,"[{'dep': 'nsubj:pass', 'lemma': 'магистраль', ...","[(аккуратный, 2)]",2
6090,LAVR / Пропитка для воздушного фильтра мотоцик...,цвет,Думал будет более липкая или же ложилась бы ра...,"[{'dep': 'ROOT', 'lemma': 'думать', 'pos': 'VE...","[(нужный, 2)]",2
6110,LAVR / Трехуровневая очистка топливной системы...,Промывка,"Промывка отличная , выкатал по инструкции , уж...","[{'dep': 'nsubj', 'lemma': 'промывка', 'pos': ...","[(надлежащий, 2)]",2


In [ ]:
def check_pos(words, pos_type="NOUN"):
    """
    Проверяет, являются ли данные слова заданной частью речи.
    
    Parameters:
    words (list): Список слов для проверки
    pos_type (str): Тип части речи для проверки (например, "NOUN", "VERB")
    
    Returns:
    list: Список булевых значений, где True указывает, что слово является заданной частью речи
    """
    results = []
    for doc in tqdm(nlp.pipe(words, batch_size=160, disable=["parser", "ner"]), total=len(words)):
        # Предполагается, что каждый документ состоит из одного токена
        token = doc[0]
        results.append(token.pos_ == pos_type and re.match(r'^[\w]+$', token.text))
    return results
grouped_text_sorted[(check_pos(grouped_text_sorted['nouns'])) |(grouped_text_sorted["nouns"] == "-")]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 562/562 [00:00<00:00, 1249.85it/s]


,product,nouns,text,text_details,lemma_counts,lemma_sum
26,3TON / Очиститель карбюратора и дроссельной за...,генераторе,"Чистил карбюратор на генераторе , всё отчистил...","[{'dep': 'ROOT', 'lemma': 'чистил', 'pos': 'VE...","[(хороший, 3)]",3
5120,GRASS / Универсальный пенный очиститель Multip...,пункте выдачи,На пункте выдачи заметила только треснувший ко...,"[{'dep': 'case', 'lemma': 'на', 'pos': 'ADP', ...","[(мелкий, 2)]",2
5013,GRASS / Универсальный пенный очиститель Multip...,коллеги,"Товар пришел быстро , целый , по рекомендациям...","[{'dep': 'nsubj', 'lemma': 'товар', 'pos': 'NO...","[(мелкий, 2)]",2
5022,GRASS / Универсальный пенный очиститель Multip...,крышка треснута,Впервые не проверила целостность на ПВЗ и вот ...,"[{'dep': 'advmod', 'lemma': 'впервые', 'pos': ...","[(негативный, 2), (неплохой, 2), (мягкий, 2), ...",10
5033,GRASS / Универсальный пенный очиститель Multip...,максимуму,"Баллона хватило на всю машину , но я в несколь...","[{'dep': 'nsubj', 'lemma': 'баллона', 'pos': '...","[(первый, 4), (похожий, 2), (второй, 2), (порш...",10
...,...,...,...,...,...,...
3060,"GRASS / Автошампунь для бесконтактной мойки, A...",водой,Ни когда не развожу с водой. Мою мойкой с встр...,"[{'dep': 'advmod', 'lemma': 'ни', 'pos': 'PART...","[(хороший, 2), (отличный, 2)]",4
3036,"GRASS / Автошампунь для бесконтактной мойки, A...",автомобиль,На втором фото после еще дополнительно протира...,"[{'dep': 'case', 'lemma': 'на', 'pos': 'ADP', ...","[(отличный, 8), (хороший, 7), (приятный, 2)]",17
3034,"GRASS / Автошампунь для бесконтактной мойки, A...",авто шампуня,"Вместо авто шампуня , положили ополаскиватель ...","[{'dep': 'case', 'lemma': 'вместо', 'pos': 'AD...","[(нормальный, 2)]",2
3015,"GRASS / Автошампунь для бесконтактной мойки, A...",Производителю респект,\n Последние время WB очень расстраивает .......,"[{'dep': 'dep', 'lemma': ' ', 'pos': 'SPACE',...","[(хороший, 4)]",4
